In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import codecs
from bs4 import BeautifulSoup
import time
import json
from multiprocessing.dummy import Pool as ThreadPool 
from multiprocessing.dummy import Lock as ThreadLock 
from multiprocessing.dummy import Value as ThreadValue
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import nltk
from sklearn.metrics import pairwise_distances
import pymorphy2
from multiprocessing.dummy import Lock
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer

In [9]:
morph = pymorphy2.MorphAnalyzer()

In [10]:
stemmerR = SnowballStemmer('russian')
stemmerE = SnowballStemmer('english')

In [11]:
def inp_for_titles(line):
    data = line.split('\t', 1)
    doc_id = int(data[0])
    if len(data[1]) == 1:
        title = ''
    else:
        title = data[1]
    doc_to_title[doc_id] = title[:-1]

In [12]:
doc_to_title = {}
with codecs.open('docs_titles.tsv', 'r', 'utf8') as f:
    next(f)
    with ThreadPool(10) as pool:
        pool.map(inp_for_titles, f)
print(len(doc_to_title))

28026


In [13]:
doc_to_title

{15731: 'ВАЗ 21213 | Замена подшипников ступицы | Нива',
 14829: 'Ваз 2107 оптом в Сочи. Сравнить цены, купить потребительские товары на Tiu.ru',
 15764: 'Купить ступица Лада калина2. Трансмиссия - переходные ступицы цена, замена, тюнинг.',
 17669: 'Классика 21010 - 21074',
 14852: 'Ступица Нива — замена подшипника своими руками',
 15458: 'ВАЗ 2110',
 14899: 'Обзор подшипников полуоси ВАЗ 2101-07, 2121,2123',
 16879: 'Купить Подшипники и ступицы FAG (Страница 23)',
 16310: 'HorsePowers — автомобильный интернет портал » Отзыв владельца ВАЗ 2121 Нива 2007 года',
 15440: 'Новости и сообщения из официальной группы Вконтакте торговой компании 33 Sport - Магазины - Тольятти',
 16242: 'Инструкция по замене подшипника передней ступицы ивеко дейли через dorognoekam.ru',
 16383: 'Ступицы - OLX.ua - страница 80',
 15580: 'маааленькая проблемка — бортжурнал Автокам 2160 ╬ 1994 года на DRIVE2',
 16840: 'Разгруженные полуоси для Нива (24 шлица 765 мм)',
 17519: 'Прошивки для нива м7.9.7 скачать - Фа

In [14]:
stop_words = set(stopwords.words(["russian", "english"]))

In [21]:
lock = Lock()

In [23]:
def Parser_for_titles(ID):
    title = (doc_to_title[ID])
    title = title.lower()
    words = [stemmerR.stem(stemmerE.stem(word)) for word in re.sub('[^a-zа-я0-9]', ' ', title).split()]
    with lock:
        vec[ID] = ' '.join([word for word in words])

In [24]:
vec = {}
with ThreadPool(10) as pool:
    pool.map(Parser_for_titles, list(range(1,28027)))

In [25]:
vec

{1: 'м б аншин центр репродукц и генетик фертимед г москв',
 2: 'перевод кив кошельк',
 3: 'проект патрул времен реабилитац духовн существ просмотр тем деньг и экономик обм и финанс деньг без процент',
 4: 'блог клуб преподаван в начальн класс порта профессиональн образован чеченск республик',
 702: 'медицинск порта узбекиста med uz дерматовенеролог рус',
 703: 'уста',
 704: 'как состав бизнес план карьер и сво бизнес',
 705: 'вариац на тем подкормк томат стр 1 томат в теплиц форум',
 706: 'зубн врач и врач стоматолог в чем разниц особен професс',
 1403: 'где в казан можн куп медилис ципер наш справк в город казан татарста',
 1404: 'первичн бесплод причин диагностик профилактик',
 1405: 'стен вконтакт',
 1406: 'экстернат больш не будет',
 1407: 'мыш 2 штук рабоч оптическ цен за 2 мыш куп в красноярск состоян б у клавиатур и мыш на интернет аукцион au ru',
 707: 'как уменьш холестерин в кров в домашн услов народн средств здоров питан и фитнес golbi',
 708: 'современ сатирическ рома лите

In [27]:
with open('titles_stemming.json', 'w', encoding='utf8') as file:
    json.dump(vec, file)

Застемил заголовки - уже прекрасно!

## Мутим фичи

In [1]:
!pip install fasttext

  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-macosx_10_12_x86_64.whl size=324287 sha256=1aca536a5f62e01a9433b11746a610136aa323a703f84c1f45af47de11095c96
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
!pip install gensim

     |████████████████████████████████| 24.2 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 117 kB 2.2 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-4.0.1-py3-none-any.whl size=108249 sha256=54b93ee534545cb3657fe4c27116c6ea53bd50ab3c37028568418f11c50e2d07
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/34/3d/14/f19c01a19c9201cdb6a76b049904d5226912569be919ad1eae
Successfully built smart-open
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


In [28]:
from gensim.models import FastText

from sklearn.metrics.pairwise import cosine_similarity

Взяли заголовки

In [30]:
sentences = []
for value in vec.values():
    sentences.append(value)

In [32]:
model_ted = FastText(sentences, size=100, window=5, min_count=5, workers=4, sg=1)

In [36]:
first = sentences[0]

In [37]:
embedding = model_ted.wv[first]

In [39]:
embedding

(100,)

In [70]:
class Import():
    def __init__(self):
        pass
    
    def import_from_file(self, file):
        df = pd.read_csv(file)
        groups = df.groupby('group_id')
        if 'target' in df.columns:
            X = np.empty(shape=(df.shape[0]), dtype=np.float)
            y = np.empty(shape=(df.shape[0]), dtype=bool)
            group_ids = np.empty(shape=(df.shape[0], ), dtype=int)
            i = 0
            print(groups.groups.items())
            return X, y, group_ids

In [71]:
importer = Import()

In [72]:
X, y, group_ids = importer.import_from_file('train_groups.csv')

dict_items([(1, Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
             92,  93,  94,  95,  96,  97,  98,  99, 100, 101],
           dtype='int64', length=102)), (2, Int64Index([102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
            115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
            128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
            141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
            154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
            167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179,
            180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192,
            193, 194, 195],
           dtype='int64')), (3, Int64Index([196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208,
            209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,
            222, 223, 224, 225, 226, 227, 228,